# How To: Compute likelihood and chi2

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from smodels.tools.simplifiedLikelihoods import LikelihoodComputer, Data

## How to compute the likelihood and chi2 from the number of events

In [3]:
# If the number of observed events, the number of expected background events,
# its error and the number of signal events and its error are known, the likelihood
# for the signal (assuming a truncated gaussian distribution for the background and signal uncertainties)
# can be computed as:
m=Data ( observed=5, backgrounds=4.2, covariance=0.71**2, third_moment=None, nsignal=.1 )
comp=LikelihoodComputer ( m )
print ('likelihood=',comp.likelihood(nsig = 10. ) )
print ('chi2=',comp.chi2( nsig=10.))

likelihood= 0.0014241306489629875
chi2= 8.425350487466552


## How to compute the likelihood and chi2 from a theory prediction

In [4]:
# In most cases one wants to compute the likelihood and chi2 for a given theory prediction computed by SModelS.
# Below we generate theory predictions and compute the likelihood and chi2 values for them
# First we import those parts of smodels that are needed for this exercise
#(We will assume the input is a SLHA file. For LHE files, use the lheDecomposer instead)
from smodels.theory import slhaDecomposer
from smodels.installation import installDirectory
from smodels.tools.physicsUnits import fb, GeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database

In [5]:
# Define the SLHA input file name
filename="inputFiles/slha/gluino_squarks.slha"

In [6]:
# Load the database, do the decomposition and compute theory predictions:
# (Look at the theory predictions HowTo to learn how to compute theory predictions)
database = Database("official")
expResults = database.getExpResults()
topList = slhaDecomposer.decompose(filename, sigcut = 0.03 * fb, doCompress=True, doInvisible=True,minmassgap = 5* GeV)
allThPredictions = [theoryPredictionsFor(exp, topList) for exp in expResults]

[SModelS:pyhfInterface] WARNING you are using pyhf v0.4.3.
[SModelS:pyhfInterface] We recommend pyhf >= 0.5.2. Please try to update pyhf ASAP!


In [7]:
# For each theory prediction, compute the corresponding likelihood and chi2 values
# (This is only possible for efficiency map-type results):
for i,thPreds in enumerate(allThPredictions):
    if not thPreds: continue #skip results with no predictions
    expID = expResults[i].globalInfo.id
    dataType = expResults[i].getValuesFor('dataType')[0]    
    for theoryPred in thPreds:
        #Compute the likelihood and chi2:
        theoryPred.computeStatistics()
        print ("\nExperimental Result: %s (%s-type)" %(expID,dataType)) #Result ID and type
        print ("Theory prediction xsec = ",theoryPred.xsection.value) #Signal xsection*efficiency*BR
        if dataType == 'efficiencyMap':
            print ('likelihood =',theoryPred.likelihood,', chi2 =',theoryPred.chi2)
        else:
            print ("(likelihood not available)")



Experimental Result: ATLAS-SUSY-2015-06 (efficiencyMap-type)
Theory prediction xsec =  4.21E-03 [pb]
likelihood = 0.00047191003417079025 , chi2 = 0.2994993563694699

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  4.91E-02 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  1.16E-02 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  7.53E-03 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  1.03E-01 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (efficiencyMap-type)
Theory prediction xsec =  9.72E-03 [pb]
likelihood = 2.5183490485185764e-09 , chi2 = 25.728996112210737

Experimental Result: ATLAS-SUSY-2016-14 (upperLimit-type)
Theory prediction xsec =  3.50E-05 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-


Experimental Result: CMS-SUS-13-007 (efficiencyMap-type)
Theory prediction xsec =  2.07E-08 [pb]
likelihood = 0.3773394259512739 , chi2 = 1.1588822770817544

Experimental Result: CMS-SUS-13-012 (upperLimit-type)
Theory prediction xsec =  6.62E-04 [pb]
(likelihood not available)

Experimental Result: CMS-SUS-13-012 (upperLimit-type)
Theory prediction xsec =  2.11E-03 [pb]
(likelihood not available)

Experimental Result: CMS-SUS-13-012 (upperLimit-type)
Theory prediction xsec =  3.07E-06 [pb]
(likelihood not available)

Experimental Result: CMS-SUS-13-012 (efficiencyMap-type)
Theory prediction xsec =  3.45E-04 [pb]
likelihood = 0.0007667344162978195 , chi2 = 6.883662496088301

Experimental Result: CMS-SUS-13-013 (upperLimit-type)
Theory prediction xsec =  3.07E-06 [pb]
(likelihood not available)

Experimental Result: CMS-SUS-13-013 (efficiencyMap-type)
Theory prediction xsec =  4.16E-08 [pb]
likelihood = 0.10977273124946735 , chi2 = 0.006572298674409005

Experimental Result: CMS-SUS-13-

In [8]:
## for an example that involves combination of signal regions and different treatment of the nuisances, 
## see the "marginalize" how to.